In [1]:
import nltk
nltk.download('punkt')
import pyfiglet
import os
import numpy as np
import pickle
import wikipedia

import import_ipynb
import preprocess_utils
import parse_utils
import ai_kw_detect
import wikt_def_parse_Wex
import wikt_def_predict_Wex
import keyword_utils


[nltk_data] Downloading package punkt to
[nltk_data]     /home/jackragless/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/jackragless/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/jackragless/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


importing Jupyter notebook from preprocess_utils.ipynb
importing Jupyter notebook from parse_utils.ipynb


[nltk_data] Downloading package benepar_en3 to
[nltk_data]     /home/jackragless/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


importing Jupyter notebook from ai_kw_detect.ipynb
importing Jupyter notebook from wikt_def_parse_Wex.ipynb
importing Jupyter notebook from wikt_def_predict_Wex.ipynb
importing Jupyter notebook from keyword_utils.ipynb


In [2]:
# pages = ['Computational Intelligence', 'Dialogflow', 'Bayesian programming', 'BabyX', 'Fuzzy agent']
# pages = ['Augmented reality-assisted surgery','Universal Scene Description','Junaio','USens','ARCore']
# for page in pages:
#     text_file = open("text_files/{}.txt".format(page), "w")
#     text_file.write(wikipedia.page(page).content)
#     text_file.close()

In [3]:
pos_association = {
    'CC':['conjunction'],
    'CD':['numeral'],
    'DT':['determiner'],
    'EX':[],
    'FW':[],
    'IN':['preposition','conjunction'],
    'JJ':['adjective'],
    'JJR':['adjective'],
    'JJS':['adjective'],
    'LS':[],
    'MD':['verb'],
    'NN':['noun','proper noun'],
    'NNS':['noun'], #PLURAL
    'NNP':['proper noun', 'noun'],
    'NNPS':['proper noun', 'noun'], #plural
    'PDT':['determiner'],
    'POS':[],
    'PRP':['pronoun'],
    'PRP$':['pronoun'],
    'RB':['adverb'],
    'RBR':['adverb'],
    'RBS':['adverb'],
    'RP':['preposition'], #unsure
    'TO':[], #unsure
    'UH':['interjection'],
    'VB':['verb'],
    'VBG':['verb'],
    'VBD':['verb'],
    'VBN':['verb'],
    'VBP':['verb'],
    'VBZ':['verb'],
    'WDT':['determiner'],
    'WP':['pronoun'],
    'WRB':['adverb'],
    
    #to deal with keyphrases
    'noun':['noun', 'proper noun'],
    'verb':['verb']
}

In [4]:
def generate_keywords(processed_text):
    
    candidate_phrases = []
    pos = []
    tok_sents = []
    
    for sent in nltk.sent_tokenize(processed_text):
        
        temp_tree = parse_utils.parseSent(sent)
        if temp_tree:
            tok_sents.append(sent[:-1])
            candidate_phrases += parse_utils.getPhraseNodes(temp_tree,[])
            pos.append( parse_utils.getWordNodes(temp_tree,[]) )
        
        
    pred_kw = ai_kw_detect.predict(tok_sents)
    
    
    FINAL_KW = []
    kw_only = []
    
    for i in range(len(pred_kw)):
        if pred_kw[i]:
            for j in range(len(pred_kw[i])):
                if len(pred_kw[i][j].split()) == 1:
                    for k in range(len(pos[i])):  
                        if pred_kw[i][j] == pos[i][k][0] and pred_kw[i][j] not in kw_only:
                            FINAL_KW.append([pos[i][k][0], pos_association[pos[i][k][1]]])
                            kw_only.append(pos[i][k][0])
                            break
                else:
                    for m in range(len(candidate_phrases)):
                        
                        if pred_kw[i][j] == candidate_phrases[m][0] and pred_kw[i][j] not in kw_only:
                            FINAL_KW.append([candidate_phrases[m][0],pos_association[candidate_phrases[m][1]]])
                            kw_only.append(candidate_phrases[m][0])
                            break
                    
        
    return FINAL_KW

In [5]:
def generate_definitions(title, clean_text, keyword_arr):
    final = []
    count = 0
    for kw in keyword_arr:
        count += 1
        print(count,'/',len(keyword_arr))
        
        if kw[0].lower() not in [kw[0].lower() for kw in final]:
            for pos in kw[1]:
                temp_def = wikt_def_predict_Wex.driver(title, clean_text, kw[0], pos)
    #             print(kw[0], '<--->', temp_def)
                if temp_def == 'invalid-term':
                    final.append([kw[0],'invalid-term'])
                elif temp_def == 'invalid-pos':
                    final.append([kw[0],'invalid-pos'])
                else:
                    final.append([kw[0],temp_def])
                    break



    return final

In [6]:
# print(pyfiglet.figlet_format("DAIC GLOGEN"),)
# print('DESCRIPTION: GLOGEN automatically generates glossaries and prepends them to given .txt files. \nENSURE: <filename>.txt == original text title.')
# answer = ''
# txt_address = ''
# while True:
#     answer = input('\nType "yes" / "no" to starting GLOGEN:\n>>>')
#     if answer.lower().startswith("y"):
#         txt_address += input('Type address where .txt files are stored. If same address as main.py press ENTER.\n>>>')
#         break
#     elif answer.lower().startswith("n"):
#         exit()
#     else:
#         print('INVALID INPUT --- TRY AGAIN.')
#         continue
# if txt_address == '':
#     txt_address = os.getcwd()

In [7]:
user_input_data = []

txt_address = '/home/jackragless/projects/github/DAIC_GLOGEN/text_files'
    
for filename in os.listdir(txt_address):
    if filename.endswith('.txt'):
        print(filename[:-4])
        orig_text = open(txt_address+'/'+filename).read()
        
        processed = preprocess_utils.clean_text(orig_text, False,False,True,False,False) 
        keywords = generate_keywords(processed)
        keyword_utils.chink(keywords)
#         keywords = keyword_utils.chink(keywords)
    
        user_input_data.append({
             'title':filename[:-4], 
             'raw_text':orig_text,
#              'processed':processed,
             'keywords':keywords,
             'glossary':generate_definitions(filename[:-4], processed, keywords)
            })

USens


  0%|          | 0/14 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

1 / 12
2 / 12
3 / 12
4 / 12
5 / 12
6 / 12
7 / 12
8 / 12
9 / 12
10 / 12


/home/jackragless/miniconda3/lib/python3.8/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/jackragless/miniconda3/lib/python3.8/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


...

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

1 / 18
2 / 18
3 / 18
4 / 18
5 / 18
6 / 18
7 / 18
8 / 18
9 / 18
10 / 18
11 / 18
12 / 18
13 / 18
14 / 18
15 / 18
16 / 18
17 / 18
18 / 18
Junaio


  0%|          | 0/15 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

1 / 23
2 / 23
3 / 23
4 / 23
5 / 23
6 / 23
7 / 23
8 / 23
9 / 23
10 / 23
11 / 23
12 / 23
13 / 23
14 / 23
15 / 23
16 / 23
17 / 23
18 / 23
19 / 23
20 / 23
21 / 23
22 / 23
23 / 23


In [8]:
user_input_data

[{'title': 'USens',
  'raw_text': "uSens, Inc. is a Silicon Valley startup founded in 2014 in San Jose, California.  The company's core team includes researchers and developers building interactive and immersive computer-vision tracking solutions. The team has extensive experience in artificial intelligence (AI), computer vision, 3D Human–computer interaction (HCI) technology and augmented reality and virtual reality. uSens has been applying computer vision and AI technologies in AR/VR, Automotive and smartphones.  \nuSens has its corporate headquarters in San Jose, California, with additional offices in China in Beijing, Hangzhou, and Shenzhen.\n\n\n== History ==\nThe co-founder of uSens, Dr. Yue Fei, is a researcher in the field of 3D user interfaces and virtual reality who earned his PhD from Rice University in Space Physics.uSens has developed products in the area of HCI, such as the Fingo Virtual Touch, selling in China. Fingo Virtual Touch uses the company’s patented Fingo techno